# ETL – Día 3  
## Transformaciones, Validaciones y Enriquecimiento de Datos

Este notebook corresponde al Día 3 del pipeline ETL y se centra en la fase
Transform, aplicando validaciones de negocio, transformaciones condicionales,
enriquecimiento de datos y generación de métricas analíticas.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Crear conjunto de datos con problemas realistas
np.random.seed(42)
n = 1000

df = pd.DataFrame({
    'id_cliente': range(1, n+1),
    'edad': np.random.normal(35, 15, n).clip(18, 80).astype(int),
    'ingresos': np.random.lognormal(10, 0.8, n),
    'gastos_mensuales': np.random.normal(2000, 500, n).clip(500, 10000),
    'categoria_cliente': np.random.choice(['A', 'B', 'C', 'D'], n),
    'fecha_registro': pd.date_range('2020-01-01', periods=n, freq='D')[:n],
    'email': [f'cliente{i}@ejemplo.com' for i in range(1, n+1)],
    'telefono': [
        f'({np.random.randint(100, 999)}){np.random.randint(100, 999)}-{np.random.randint(1000, 9999)}'
        for _ in range(n)
    ]
})

df.head()

,id_cliente,edad,ingresos,gastos_mensuales,categoria_cliente,fecha_registro,email,telefono
0,1,42,67473.104747,1662.410863,B,2020-01-01,cliente1@ejemplo.com,(138)588-4715
1,2,32,46152.524769,1927.740665,D,2020-01-02,cliente2@ejemplo.com,(641)769-2301
2,3,44,23102.688983,1603.790040,B,2020-01-03,cliente3@ejemplo.com,(302)165-3155
3,4,57,13127.316641,1846.019235,B,2020-01-04,cliente4@ejemplo.com,(367)321-1692
4,5,31,38506.358054,1053.192667,A,2020-01-05,cliente5@ejemplo.com,(354)554-7396


In [3]:
# Introducir errores intencionalmente
error_indices = np.random.choice(n, 50, replace=False)

# Edades inválidas
df.loc[error_indices[:20], 'edad'] = np.random.choice([-5, 150, np.nan], 20)

# Ingresos negativos
df.loc[error_indices[20:35], 'ingresos'] = -1000

# Gastos mayores que ingresos
df.loc[error_indices[35:], 'gastos_mensuales'] = (
    df.loc[error_indices[35:], 'ingresos'] * 2
)

df.loc[error_indices].head()

,id_cliente,edad,ingresos,gastos_mensuales,categoria_cliente,fecha_registro,email,telefono
780,781,150.0,7150.560535,2437.258537,B,2022-02-19,cliente781@ejemplo.com,(377)662-6782
485,486,150.0,46447.255873,2429.793943,B,2021-04-30,cliente486@ejemplo.com,(578)919-4516
679,680,-5.0,15594.970694,1868.275892,C,2021-11-10,cliente680@ejemplo.com,(829)214-2603
351,352,150.0,5210.419501,2308.502975,C,2020-12-17,cliente352@ejemplo.com,(939)717-1471
615,616,150.0,283363.570715,2334.170244,D,2021-09-07,cliente616@ejemplo.com,(163)128-3419


In [4]:
# Validar edades
df['edad_valida'] = df['edad'].apply(
    lambda x: True if pd.notna(x) and 18 <= x <= 80 else False
)

# Marcar edades inválidas como NaN
df.loc[~df['edad_valida'], 'edad'] = np.nan

# Ver ejemplos de corrección
df.loc[df['edad_valida'] == False, ['id_cliente', 'edad', 'edad_valida']].head(10)

,id_cliente,edad,edad_valida
49,50,NaN,False
120,121,NaN,False
226,227,NaN,False
299,300,NaN,False
321,322,NaN,False
351,352,NaN,False
357,358,NaN,False
364,365,NaN,False
396,397,NaN,False
485,486,NaN,False


In [5]:
# Validar ingresos (no negativos)
df[df['ingresos'] < 0][['id_cliente', 'ingresos']].head(10)

,id_cliente,ingresos
99,100,-1000.0
232,233,-1000.0
363,364,-1000.0
525,526,-1000.0
539,540,-1000.0
643,644,-1000.0
644,645,-1000.0
761,762,-1000.0
765,766,-1000.0
767,768,-1000.0


In [6]:
df.loc[df['ingresos'] < 0, 'ingresos'] = np.nan

In [7]:
df['edad_valida'].value_counts()

edad_valida
True     980
False     20
Name: count, dtype: int64

In [8]:
df['ingreso_valido'].value_counts()

KeyError: 'ingreso_valido'

In [9]:
df['ingreso_valido'] = df['ingresos'].apply(
    lambda x: True if pd.notna(x) and x > 0 else False
)

In [10]:
df.loc[df['ingreso_valido'] == False, ['id_cliente', 'ingresos', 'ingreso_valido']].head(10)

,id_cliente,ingresos,ingreso_valido
99,100,NaN,False
232,233,NaN,False
363,364,NaN,False
525,526,NaN,False
539,540,NaN,False
643,644,NaN,False
644,645,NaN,False
761,762,NaN,False
765,766,NaN,False
767,768,NaN,False


In [11]:
df['ingreso_valido'].value_counts()

ingreso_valido
True     985
False     15
Name: count, dtype: int64

In [12]:
df['ratio_gasto_ingreso'] = df['gastos_mensuales'] / df['ingresos']

In [13]:
df['gasto_valido'] = df['ratio_gasto_ingreso'] <= 1

In [14]:
df['gasto_valido'].value_counts(dropna=False)

gasto_valido
True     969
False     31
Name: count, dtype: int64

In [15]:
mask_gasto_invalido = df['ratio_gasto_ingreso'] > 1

df.loc[mask_gasto_invalido, 'gastos_mensuales'] = (
    df.loc[mask_gasto_invalido, 'ingresos'] * 0.8
)

In [16]:
df['ratio_gasto_ingreso'] = df['gastos_mensuales'] / df['ingresos']

In [17]:
df['ratio_gasto_ingreso'].describe()

count    985.000000
mean       0.127853
std        0.142430
min        0.005699
25%        0.047627
50%        0.085558
75%        0.143558
max        0.937917
Name: ratio_gasto_ingreso, dtype: float64

In [18]:
df.loc[mask_gasto_invalido, 
       ['id_cliente', 'ingresos', 'gastos_mensuales', 'ratio_gasto_ingreso']].head(10)

,id_cliente,ingresos,gastos_mensuales,ratio_gasto_ingreso
39,40,50903.076466,40722.461173,0.8
68,69,15151.260957,12121.008765,0.8
119,120,51048.095524,40838.476419,0.8
182,183,45919.873895,36735.899116,0.8
249,250,6344.118759,5075.295007,0.8
255,256,8368.979103,6695.183282,0.8
345,346,6714.882598,5371.906078,0.8
429,430,17651.028985,14120.823188,0.8
469,470,31435.155093,25148.124074,0.8
546,547,9909.920932,7927.936745,0.8


In [19]:
# Categorizar por edad
df['grupo_edad'] = pd.cut(
    df['edad'],
    bins=[18, 25, 35, 50, 80],
    labels=['Joven', 'Adulto_Joven', 'Adulto', 'Senior']
)

In [20]:
# Calcular capacidad de ahorro
df['capacidad_ahorro'] = df['ingresos'] - df['gastos_mensuales']
df['ratio_ahorro'] = df['capacidad_ahorro'] / df['ingresos']

In [21]:
# Clasificar capacidad financiera
df['clasificacion_financiera'] = np.where(
    df['ratio_ahorro'] > 0.3, 'Ahorra_Mucho',
    np.where(
        df['ratio_ahorro'] > 0.1, 'Ahorra_Poco',
        np.where(
            df['ratio_ahorro'] > 0, 'Equilibra',
            'Deficit'
        )
    )
)

In [22]:
# Extraer información del teléfono
df['codigo_area'] = df['telefono'].str.extract(r'\((\d{3})\)')

In [23]:
# Calcular antigüedad
df['antiguedad_dias'] = (pd.Timestamp.now() - df['fecha_registro']).dt.days
df['antiguedad_meses'] = df['antiguedad_dias'] // 30

In [24]:
df['grupo_edad'].value_counts()

grupo_edad
Adulto          336
Adulto_Joven    249
Senior          136
Joven           132
Name: count, dtype: int64

In [25]:
# Métricas por grupo de edad
metricas_edad = df.groupby('grupo_edad').agg({
    'ingresos': ['mean', 'median', 'std'],
    'capacidad_ahorro': 'mean',
    'ratio_ahorro': 'mean'
}).round(2)

C:\Users\vylo0\AppData\Local\Temp\ipykernel_16292\885204534.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  metricas_edad = df.groupby('grupo_edad').agg({


In [26]:
metricas_edad = df.groupby('grupo_edad', observed=True).agg({
    'ingresos': ['mean', 'median', 'std'],
    'capacidad_ahorro': 'mean',
    'ratio_ahorro': 'mean'
}).round(2)

In [27]:
print("Métricas por grupo de edad:")
print(metricas_edad)

Métricas por grupo de edad:
              ingresos                     capacidad_ahorro ratio_ahorro
                  mean    median       std             mean         mean
grupo_edad                                                              
Joven         32954.67  26454.09  25962.62         30926.97         0.88
Adulto_Joven  31465.15  23640.71  25554.76         29336.67         0.87
Adulto        31504.19  21448.63  32238.73         29145.95         0.87
Senior        31016.96  22286.82  26921.09         28876.55         0.88


In [28]:
# Resumen de validaciones
resumen_validacion = {
    'total_registros': len(df),
    'edades_invalidas': (~df['edad_valida']).sum(),
    'ingresos_negativos_corregidos': (df['ingresos'].isna()).sum(),
    'registros_procesados': len(df)
}

print("\nResumen de validación:")
for clave, valor in resumen_validacion.items():
    print(f"{clave}: {valor}")


Resumen de validación:
total_registros: 1000
edades_invalidas: 20
ingresos_negativos_corregidos: 15
registros_procesados: 1000


In [29]:
metricas_edad.to_excel('metricas_por_grupo_edad.xlsx')

In [30]:
metricas_edad.to_sql('metricas_edad', conn)

NameError: name 'conn' is not defined

In [31]:
df_raw = df.copy()

In [32]:
with pd.ExcelWriter('evidencia_transformacion_dia3.xlsx', engine='openpyxl') as writer:
    
    # Datos originales
    df_raw.to_excel(writer, sheet_name='datos_originales', index=False)
    
    # Datos transformados
    df.to_excel(writer, sheet_name='datos_transformados', index=False)
    
    # Métricas agregadas
    metricas_edad.to_excel(writer, sheet_name='metricas_por_edad')
    
    # Resumen de validaciones
    resumen_validacion_df.to_excel(writer, sheet_name='resumen_validacion')

NameError: name 'resumen_validacion_df' is not defined

In [33]:
resumen_validacion_df = pd.DataFrame(
    resumen_validacion.items(),
    columns=['metricas', 'valor']
)

In [34]:
with pd.ExcelWriter('evidencia_transformacion_dia3.xlsx', engine='openpyxl') as writer:
    
    # Datos originales
    df_raw.to_excel(writer, sheet_name='datos_originales', index=False)
    
    # Datos transformados
    df.to_excel(writer, sheet_name='datos_transformados', index=False)
    
    # Métricas agregadas
    metricas_edad.to_excel(writer, sheet_name='metricas_por_edad')
    
    # Resumen de validaciones
    resumen_validacion_df.to_excel(writer, sheet_name='resumen_validacion', 

_IncompleteInputError: incomplete input (1801682156.py, line 13)

In [35]:
with pd.ExcelWriter('evidencia_transformacion_dia3.xlsx', engine='openpyxl') as writer:
    
    # Datos originales
    df_raw.to_excel(writer, sheet_name='datos_originales', index=False)
    
    # Datos transformados
    df.to_excel(writer, sheet_name='datos_transformados', index=False)
    
    # Métricas agregadas
    metricas_edad.to_excel(writer, sheet_name='metricas_por_edad')
    
    # Resumen de validaciones
    resumen_validacion_df.to_excel(writer, sheet_name='resumen_validacion', index=False)

In [37]:
comparacion = df_raw.loc[
    df_raw.index.isin(error_indices),
    ['id_cliente', 'edad', 'ingresos', 'gastos_mensuales']
].copy()

comparacion[['edad_corr', 'ingresos_corr', 'gastos_corr']] = df.loc[
    comparacion.index,
    ['edad', 'ingresos', 'gastos_mensuales']
]

comparacion.to_excel(writer, sheet_name='antes_vs_despues', index=False)

In [38]:
with pd.ExcelWriter('evidencia_dia3_transformacion.xlsx', engine='xlsxwriter') as writer:

    comparacion = df_raw.loc[
        indices_afectados,
        ['id_cliente', 'edad', 'ingresos', 'gastos_mensuales']
    ].copy()

    comparacion[['edad_corr', 'ingresos_corr', 'gastos_corr']] = df.loc[
        comparacion.index,
        ['edad', 'ingresos', 'gastos_mensuales']
    ]

    comparacion.to_excel(writer, sheet_name='antes_vs_despues', index=False)

ModuleNotFoundError: No module named 'xlsxwriter'

In [39]:
with pd.ExcelWriter(
    'evidencia_dia3_transformacion.xlsx',
    engine='openpyxl'
) as writer:

    comparacion.to_excel(
        writer,
        sheet_name='antes_vs_despues',
        index=False
    )

    metricas_edad.to_excel(
        writer,
        sheet_name='metricas_por_edad'
    )

    resumen_validacion_df.to_excel(
        writer,
        sheet_name='resumen_validacion',
        index=False
    )

print("✅ Evidencia del Día 3 generada correctamente")

✅ Evidencia del Día 3 generada correctamente


In [40]:
df_raw = df.copy()

In [41]:
import pandas as pd

# Crear DataFrame del resumen de validación
resumen_validacion_df = pd.DataFrame(
    list(resumen_validacion.items()),
    columns=['Regla', 'Cantidad']
)

# Seleccionar filas afectadas para antes vs después
comparacion = df_raw.loc[
    df_raw.index.isin(error_indices),
    ['id_cliente', 'edad', 'ingresos', 'gastos_mensuales']
].copy()

comparacion[['edad_corr', 'ingresos_corr', 'gastos_corr']] = df.loc[
    comparacion.index,
    ['edad', 'ingresos', 'gastos_mensuales']
]

# Crear archivo Excel final
with pd.ExcelWriter(
    'evidencia_ETL_dia3_transformaciones.xlsx',
    engine='openpyxl'
) as writer:

    df_raw.to_excel(writer, sheet_name='datos_originales', index=False)
    df.to_excel(writer, sheet_name='datos_transformados', index=False)
    comparacion.to_excel(writer, sheet_name='antes_vs_despues', index=False)
    metricas_edad.to_excel(writer, sheet_name='metricas_por_edad')
    resumen_validacion_df.to_excel(writer, sheet_name='resumen_validacion', index=False)

print("✅ Evidencia del Día 3 generada correctamente")

✅ Evidencia del Día 3 generada correctamente
